##### Tableau dashboard data download 

In [ ]:
from tableau_api_lib import TableauServerConnection
from tableau_api_lib.utils import querying
from tableau_api_lib.utils.common import flatten_dict_column
import pandas as pd

tableau_server_config = {
        'my_env': {
                'server': 'https://******.*****.com',
                'api_version': "3.6",
                'username': '****',
                'password': '***@123',
                'site_name': 'Default',
                'site_url': ''
        }
}
conn = TableauServerConnection(tableau_server_config, env='my_env')
conn.sign_in()

In [ ]:
site_views_df = querying.get_views_dataframe(conn)
site_views_detailed_df = flatten_dict_column(site_views_df, keys=['name', 'id'], col_name='workbook')
site_views_detailed_df

In [ ]:
relevant_views_df = site_views_detailed_df[site_views_detailed_df['workbook_name'] == 'More than 80 repledges during 30 days']
relevant_views_df

In [ ]:
visual_c_id = relevant_views_df[relevant_views_df['name'] == 'WorkbookMore than 80 repledges during 30 days']['id'].to_list()[0]
visual_c_id

In [ ]:
view_data_raw = querying.get_view_data_dataframe(conn, view_id=visual_c_id)
df = pd.DataFrame(view_data_raw)
ddf = df.pivot_table(index=['CUST_ID'],  columns='Measure Names', values=['Measure Values'], aggfunc='first')

In [ ]:
data = pd.merge(ddf, df, on="CUST_ID", how="left")
s = data.drop_duplicates('CUST_ID', keep='last')
s.drop(['Measure Names', 'Measure Values','SETTLED_PL_COUNT'], axis=1, inplace=True)
s.to_excel("Data.xlsx", index=False)

##### Data Insert part Create table and then insert

In [ ]:
import cx_Oracle
import pandas as pd
data = pd.read_excel(r"C:\Users\356285\Desktop\TableauPythpon\Data.xlsx")
conn = cx_Oracle.connect("****", "****$1234", "***")
cursor = conn.cursor()   

In [ ]:
sql = """CREATE TABLE Tableau_AML_80123 
    (CUST_ID varchar2(50) NOT NULL,
     INV_COUNT varchar2(50) NOT NULL, 
     LIVE_PL_AMOUNT varchar2(50) NOT NULL,
     LIVE_PL_COUNT varchar2(50) NOT NULL,
     SETTLED_PL_AMOUNT varchar2(50) NOT NULL,  
     SETTLED_PL_COUNT varchar2(50) NOT NULL
     )  """

cursor.execute(sql)

In [ ]:
rec = data.astype(str).values.tolist()

for i in range(len(rec)):
    cursor.prepare("INSERT INTO Tableau_AML_80123 VALUES (:1,:2,:3,:4,:5,:6)")
    cursor.executemany(None,([rec[i]]))

conn.commit()
conn.close

In [ ]:
# rec[:1]

In [ ]:
import cx_Oracle
import pandas
 
sql = "select * from Tableau_AML_80"

 
conn = cx_Oracle.connect("*******", "****$1234", "****")
 
cursor = conn.cursor()
cursor.execute(sql)
df = pandas.read_sql(sql,con=conn)

In [ ]:
# sql = 'DROP TABLE Tableau_AML_80'
# conn = cx_Oracle.connect("kpmg", "Asd$1234", "HISTDBOCI")
 
# cursor = conn.cursor()
# cursor.execute(sql)